# DASCore Basics
This notebook introduces the basics of DASCore. It is a shortened version of the [DASCore tutorial](https://dascore.org/tutorial/concepts.html).

In [ ]:
import dascore as dc  # the idiomatic dascore import

## DASCore concepts
This section introduces a few important ideas before diving in to DASCore usage.

### Dates and Times

First, DASCore uses [numpy date/time constructs](https://numpy.org/devdocs/reference/arrays.datetime.html) for representing times and time offsets. These can be created directly using `numpy.datetime64` but dascore provides a bit more flexibility with `to_datetime64`. DASCore also enforces that each time construct has ns precision.

Here are a few examples:

In [ ]:
from dascore.utils.time import to_datetime64, to_timedelta64, to_float

In [ ]:
# The best way to init a time is with an ISO 8601 string
time_1 = to_datetime64("2017-09-18T08:02:10.0123")
time_2 = to_datetime64("2020-01-03T12:12:12.0213")
time_3 = to_datetime64(1_000_000_000.0)

print(time_1, time_2, time_3)

In [ ]:
# The differences of datetimes is a time delta
timedelta_1 = time_2 - time_1
print(timedelta_1)

In [ ]:
# Nanoseconds is often not so readable. These can be converted to floats by dividing
# by anther timedelta64
diff_seconds = timedelta_1 / to_timedelta64(1)
diff_hours = timedelta_1 / to_timedelta64(3600)
diff_days = timedelta_1 / to_timedelta64("1 day")
print(diff_days, diff_hours, diff_seconds)

In [ ]:
# Or any time thing can be converted to a float
diff_seconds = to_float(diff_seconds)
time_1_float = to_float(time_1)

### Units
DASCore provides first class (or at least economy plus) support for units using the [pint library](https://pint.readthedocs.io/en/stable/). These can be used in many dascore functions to help avoid unit conversion errors. 

In [ ]:
from dascore.units import get_quantity, m, ft

meters = get_quantity("meters")

# Now meters should be equal to 1 meter.
assert meters == 1 * m

# Convert 10 meters to ft.
ten_m = meters * 10
print(ten_m.to(ft))


In [ ]:
# get_quantity can handle a lot of complexity!
quantity = get_quantity("10 * (PI / 10^3) (millifurlongs)/(tesla)")
print(quantity)

## Patches and Spools

The two main data containers in DASCore are the `Patch` and the `Spool`. The `Patch` is a contiguous n-dimensional array (DFOS data) with associated metadata, and the `Spool` manages a collection of `Patches`. Conceptually, it looks like this:

![](https://dascore.org/_static/patch_n_spool.png)

## Patches

In [ ]:
# The get_example_patch function is useful for loading example/test patches.
patch = dc.get_example_patch("example_event_2")

In [ ]:
# The patch str rep. provides a summary
print(patch)

In [ ]:
# The viz namespace has plotting routines
patch.viz.waterfall();

### Patch Components

The patch is composed of:
- data: the array of measurements
- coords: The coordinates of each dimension (plus others)
- attrs: the non-coordinate metdata


In [ ]:
print(patch.dims)  # A list of dimension names

In [ ]:
print(patch.coords) # The coordinates and their labels. 

In [ ]:
# The values of the distance dimension
dist = patch.get_array("distance")
# The value of the time dimension in seconds
time_s = to_float(patch.get_array("distance"))

### Trimming and sub-selection
`select` is used to trim patches. For example, to zoom in on the down-going reflection.

In [ ]:
trimmed = patch.select(time=(.04, .09), distance =(600,800))

In [ ]:
trimmed.viz.waterfall();

### Modifying Patches
Patches are immutable, meaning they cant (or at least shouldn't) be modified. This allow patch compontents to be safely shared among patches, and makes parallel processing simpler. Here are a few examples of making new patches from existing components.

In [ ]:
# A patch with all the same metadata but different data. Note the original patch is unchanged.
patch_new_data = patch.new(data=patch.data * 10)

In [ ]:
# A patch with an updated attribute (eg adding instrument)
patch_new_attribute = patch.update_attrs(instrument_id="R2D2")
print(patch_new_attribute.attrs.instrument_id)

In [ ]:
# Rename distance to depth in new patch.
patch_new_coord = patch.rename_coords(distance="depth")
print(patch_new_coord.dims)

In [ ]:
# Add a new coordinate associated with the distance dimension
distance = patch.get_array("distance")
patch_new_coord = patch.update_coords(distancia=("distance", distance*30))


In [ ]:
print(patch_new_coord)

# Spool
As stated above, spools manage a group of patches. They can be initialized in several different ways including: 
- from in-memory patches
- from a single file
- from a directory of DAS files

In [ ]:

in_memory_spool = dc.get_example_spool("diverse_das")

# save patches to disk
das_folder_path = dc.examples.spool_to_directory(in_memory_spool)
das_file_path = next(das_folder_path.glob("*.hdf5"))


In [ ]:
# From a patch or list of patches
spool = dc.spool([patch])

In [ ]:
# From a single file
spool = dc.spool(das_file_path)

In [ ]:
# From a directory of files
# Update will create an index of the contents for fast querying/access
spool = dc.spool(das_folder_path).update()

In [ ]:
print(spool)

In [ ]:
# get contents of spool as a dataframe
contents_df = spool.get_contents()
contents_df.head()

### Accessing Patches

Patches are retrieved using iteration or indexing

In [ ]:
first_patch = spool[0]
last_patch = spool[-1]

In [ ]:
for patch in spool:
    ...    

In [ ]:
# spools can also be sliced (sub-indexed)
sub = spool[1:-1]

### Selecting

`Spool` contents can be select (filtered) with `Spool.select`

In [ ]:
# Return a spool with patches that end before 1990
sub_spool = spool.select(time=(..., '1990-01-01'))
print(sub_spool)

In [ ]:
# Return a spool with patches whose station attribute is "wayout"
sub_spool = spool.select(station="wayout")
print(sub_spool)

In [ ]:
# Return a spool with patches whose tags meets a unix-style match string
sub_spool = spool.select(tag="*dom")
print(sub_spool)

### Chunking
`Spool.chunk` is used to merge contiguous/overlapping patches or create patches of new sizes.

In [ ]:
# Chunk spool for 3 second increments with 1 second overlaps
# and keep any segements at the end that don't have the full 3 seconds.
subspool = spool.chunk(time=3, overlap=1, keep_partial=True)

# Merge all contiguous segments along time dimension.
merged_spool = spool.chunk(time=None)